In [83]:
import os
from os.path import isfile, join
import logging
import re

In [129]:
markupStart = '[['
markupEnd = ']]'
rePattern = re.compile('\[\[[^\[\]]*\]\]')

In [ ]:
def getRegexMatches(fileText):
    matches = list()
    for i,m in enumerate(rePattern.finditer(fileText)):
        matchedText = m.group().strip(markupStart).strip(markupEnd)
        matches.append([m.start()-i*4,len(matchedText),matchedText])
    return matches

def getUniqueNames(matches):
    uniqueNames = set()
    for _,_,name in matches:
        for partName in name.split(' '):
            if(partName.endswith('\'s')):
                partName = partName[:-2]
            uniqueNames.add(partName)
    return uniqueNames

In [260]:
def validateSquareBrackets(fileText,fileName):
    markupStartCount = fileText.count(markupStart)
    markupEndCount = fileText.count(markupEnd)
    if markupStartCount != markupEndCount:
        logging.error (" ValidateSquareBrackets failed: {} markupStartCount: {} markupEndCount {}"
               .format(fileName.split('/')[-1],markupStartCount,markupEndCount))

def validateOccurences(fileText,fileName):
    matches = getRegexMatches(fileText)
    matchesDict = dict()
    
    for i,_,name in matches:
        currStartOffset = 0;
        for partName in name.split(' '):
            currStartIndex = i + currStartOffset;
            currStartOffset+= len(partName)+1
            if(partName.endswith('\'s')):
                partName = partName[:-2]
            matchesDict[currStartIndex] = partName

    uniqueNames = getUniqueNames(matches)
    fileTextOrig = fileText.replace(markupStart,'').replace(markupEnd,'')
    for un in sorted(uniqueNames):
        
        occurences = [m.start() for m in re.finditer(un, fileTextOrig)]
        for occ in occurences:
            if occ not in matchesDict:
                logging.error(" validateOccurences {} : {} not marked at location {}".format(fileName.split('/')[-1],un,occ))
            else:
                if matchesDict[occ]!=un :
                    logging.error(" validateOccurences {} : {} invalid match at location {}".format(fileName.split('/')[-1],un,occ))
            

def validateFile(fileName):
    fileText = open(fileName).read()
    validateSquareBrackets(fileText,fileName)
    validateOccurences(fileText,fileName)

        
    

In [261]:
folderNames = ['Abhinav']
folderPath = '../dataset_markup/'

for folderName in sorted(folderNames):
    folderName = join(folderPath,folderName)
    for fileName in sorted(os.listdir(folderName)):
        fileName = join(folderName,fileName)
        if fileName.endswith('.txt') and isfile(fileName):
            validateFile(fileName)

ERROR:root: validateOccurences 001.txt : Pugh not marked at location 1269
ERROR:root: validateOccurences 005.txt : van not marked at location 327
ERROR:root: validateOccurences 013.txt : de not marked at location 624
ERROR:root: validateOccurences 013.txt : de not marked at location 678
ERROR:root: validateOccurences 013.txt : de not marked at location 929
ERROR:root: validateOccurences 020.txt : Gethin not marked at location 1806
ERROR:root: validateOccurences 020.txt : Jones not marked at location 1319
ERROR:root: validateOccurences 020.txt : Mefin not marked at location 1658
ERROR:root: validateOccurences 024.txt : Dane not marked at location 1229
ERROR:root: validateOccurences 027.txt : Chester not marked at location 15
ERROR:root: validateOccurences 027.txt : Chester not marked at location 55
ERROR:root: validateOccurences 027.txt : Chester not marked at location 152
ERROR:root: validateOccurences 027.txt : Chester not marked at location 577
ERROR:root: validateOccurences 027.txt 